<a href="https://colab.research.google.com/github/JavalVyas2000/Estimation-of-RUL/blob/main/Estimation_of_RUL_using_Fourier_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing all the prerequisited for connecting with the drive.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Installing the python libraries that we may use
import numpy as np
import pylab as pl
from numpy import fft

In [ ]:
# Organizing data well. 
data4=pd.read_excel('gdrive/MyDrive/RUL data/B0007_discap.xlsx')
cycle_no=[]
p=int(data4['Cycle'][data4.shape[0]-1])
for i in range(p):
  cycle_no.append(i+1)
Discap=[None]*len(cycle_no)
for i in range(data4.shape[0]-1):
  Discap[int(data4['Cycle'][i])]=data4['Discap'][i]

In [ ]:
# Taking the fft and making an extrapolation of the data
def fourierExtrapolation(x, n_predict):
    n = x.size
    n_harm = 100000                 # number of harmonics in model
    t = np.arange(0, n)
    p = np.polyfit(t, x, 1)         # find linear trend in x
    x_notrend = x - p[0] * t        # detrended x
    x_freqdom = fft.fft(x_notrend)  # detrended x in frequency domain
    f = fft.fftfreq(n)              # frequencies
    indexes = list(range(n))
    # sort indexes by frequency, lower -> higher
    indexes.sort(key = lambda i: np.absolute(f[i]))
 
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        ampli = np.absolute(x_freqdom[i]) / n   # amplitude
        phase = np.angle(x_freqdom[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig + p[0] * t
    
x = df71
n_predict = test7.size
extrapolation = fourierExtrapolation(x, n_predict)
plt.figure(figsize=(16, 8))
pl.plot(np.arange(0, extrapolation.size), extrapolation, 'r', label = 'Forecast', linewidth = 4)
pl.plot(np.arange(0, x.size), x, 'y', label = 'train', linewidth = 4)
plt.plot(np.arange(x.size,x.size+test7.size),test7,label='test')
plt.xlabel("Cycle_no")
plt.ylabel("Discap")
plt.title("B0007")
pl.legend()
pl.show()

In [ ]:
# evaluation metrics - RMSE, MSE, MAPE, MAE
import math
RMSE7 = math.sqrt(mean_squared_error(test7, extrapolation[-n_predict:]))
print('Test Score: %.4f RMSE' % (RMSE7))
MSE7=mean_squared_error(test7, extrapolation[-n_predict:])
print('Test Score: %.4f RMSE' % (MSE7))
pred=np.array(extrapolation[-n_predict:])
actual=np.array(test7)
MAPE7=np.mean(np.abs((actual-pred)/actual))*100
print('Test Score: %.4f MAPE' % (MAPE7))
MAE7=np.mean(np.abs((actual-pred)))
print('Test Score: %.4f MAPE' % (MAE7))